# 1. Data Preparation
Below is our data-preprocessing code. To run the code, we need to upload **ALL four CSV files** included in our data.zip on Github via the uploaded part in the order of **(COVID - Population - WBData1 - WBData2)**.

In [20]:
import numpy as np
import pandas as pd
import altair as alt
from google.colab import files
uploaded = files.upload()

Saving covid19_world.csv to covid19_world (1).csv
Saving Population.csv to Population.csv
Saving WBData1.csv to WBData1 (1).csv
Saving WBData2.csv to WBData2 (1).csv


In [21]:
# Set-up
COVID = list(uploaded.keys())[0]
population = list(uploaded.keys())[1]
WB1 = list(uploaded.keys())[2]
WB2 = list(uploaded.keys())[3]

## **1.1** COVID-19 Weekly Data
Below shows the data wrangling and cleaning process for our visualization tasks on exploring the time trend of COVID-19 data.

In [36]:
# Read in COVID data
covid_df = pd.read_csv(COVID, header=0)

# Collapse provincial data into country-level data
country_level_data = covid_df.groupby(['ObservationDate', 'Country_Region']).sum().reset_index()
# Drop irrelevant columns & Rename
country_level_data = country_level_data.drop(columns=['Province_State'])
country_level_data = country_level_data.rename(columns={'ObservationDate': 'Date'})

In [38]:
# Convert Date column to Datetime
country_level_data['Date'] = pd.to_datetime(country_level_data['Date'])

# Convert to mmwr_week
country_level_data['MMWR_week'] = country_level_data['Date'].dt.isocalendar().week

# Convert mmwr_week back to a start date
country_level_data['Week_Start_Date'] = country_level_data['Date'] - pd.to_timedelta(country_level_data['Date'].dt.weekday, unit='d')

# Sum-up to obtain weekly data
weekly_data = country_level_data.groupby(['Week_Start_Date', 'MMWR_week', 'Country_Region']).agg({
    'Confirmed': 'sum',
    'Deaths': 'sum',
    'Recovered': 'sum',
    'Active': 'sum'
}).reset_index()

In [39]:
# Read population data
pop = pd.read_csv(population, header=0)

# Population data manipulation
# Keep only relevant columns & Rename for our convenience
columns_to_keep = ['Country (or dependency)', 'Population (2020)', 'Density (P/Km²)']
pop_df = pop[columns_to_keep]
pop_df = pop_df.rename(columns={'Country (or dependency)': 'Country_Region',
                                "Population (2020)" : "Population"})

# Merge the dataset
weekly_data = pd.merge(pop_df, weekly_data, left_on='Country_Region', right_on='Country_Region', how='left')

In [41]:
weekly_data.head()

,Country_Region,Population,Density (P/Km²),Week_Start_Date,MMWR_week,Confirmed,Deaths,Recovered,Active
0,Afghanistan,38928346,60,2020-02-24,9,13.0,0.0,0.0,13.0
1,Afghanistan,38928346,60,2020-03-02,10,8.0,0.0,0.0,8.0
2,Afghanistan,38928346,60,2020-03-09,11,53.0,0.0,0.0,53.0
3,Afghanistan,38928346,60,2020-03-16,12,175.0,1.0,7.0,167.0
4,Afghanistan,38928346,60,2020-03-23,13,632.0,20.0,12.0,600.0


In [51]:
# Downloading data
weekly_data.to_csv('Weekly Data.csv', index=False)
files.download('Weekly Data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **1.2** COVID-19 & Socioeconomic Data
Below shows the data wrangling and cleaning process for our visualization tasks on exploring the relationship between COVID-19 and socioeconomic factors.

In [42]:
# Collapse into annual data for Section 2 plots
annual_data = country_level_data.groupby(['Country_Region']).agg({
    'Confirmed': 'sum',
    'Deaths': 'sum',
    'Recovered': 'sum',
    'Active': 'sum'
}).reset_index()

# Merge the dataset
annual_data = pd.merge(pop_df, annual_data, left_on='Country_Region', right_on='Country_Region', how='left')

In [44]:
# Take a look at annual data
annual_data.head()

,Country_Region,Population,Density (P/Km²),Confirmed,Deaths,Recovered,Active
0,Afghanistan,38928346,60,8370341.0,292432.0,5904418.0,2165900.0
1,Albania,2877797,105,3693101.0,87375.0,1980336.0,1625390.0
2,Algeria,43851044,18,10439282.0,372954.0,7041511.0,3024717.0
3,American Samoa,55191,276,NaN,NaN,NaN,NaN
4,Andorra,77265,164,678294.0,15333.0,550059.0,112902.0


In [45]:
# Read in the world bank data
df1 = pd.read_csv(WB1, header=0, encoding='ISO-8859-1',on_bad_lines='skip')
df2 = pd.read_csv(WB2, header=0, encoding='ISO-8859-1',on_bad_lines='skip')

# Slice off irrelevant rows by stopping at the last country Zimbabwe
index_zw1 = df1[df1['Country Code'] == 'ZWE'].index[-1]
index_zw2 = df2[df2['Country Code'] == 'ZWE'].index[-1]
df1 = df1.iloc[:index_zw1+1]
df2 = df2.iloc[:index_zw2+1]

# Merge datasets
merged_df = pd.concat([df1, df2], axis=0)
# Drop irrelevant columns & Rename
merged_df = merged_df.drop(columns=['Series Code','Country Code'])
merged_df = merged_df.rename(columns={'2019 [YR2019]': 'Data'})

# Pivot to wider table
socioeco_df = pd.pivot_table(merged_df, values='Data', index='Country Name', columns='Series Name', aggfunc='first')
socioeco_df.reset_index(inplace=True)

# Replace placeholder with NaN
socioeco_df.replace("..", np.nan, inplace=True)
socioeco_df.columns.name = None

In [46]:
# We noticed that there are inconsistencies in country names that cannot be resolved by text similarity algorithms
# Match totally different country names prior to merging
socioeco_df.loc[socioeco_df['Country Name'] == 'American Samoa', 'Country Name'] = 'Samoa'
socioeco_df.loc[socioeco_df['Country Name'] == 'Bahamas, The', 'Country Name'] = 'Bahamas'
socioeco_df.loc[socioeco_df['Country Name'] == 'Brunei Darussalam', 'Country Name'] = 'Brunei'
socioeco_df.loc[socioeco_df['Country Name'] == 'Congo, Dem. Rep.', 'Country Name'] = 'Congo (Kinshasa)'
socioeco_df.loc[socioeco_df['Country Name'] == 'Congo, Rep.', 'Country Name'] = 'Congo (Brazzaville)'
socioeco_df.loc[socioeco_df['Country Name'] == 'Egypt, Arab Rep.', 'Country Name'] = 'Egypt'
socioeco_df.loc[socioeco_df['Country Name'] == 'Gambia, The', 'Country Name'] = 'Gambia'
socioeco_df.loc[socioeco_df['Country Name'] == 'Hong Kong SAR, China', 'Country Name'] = 'Hong Kong'
socioeco_df.loc[socioeco_df['Country Name'] == 'Iran, Islamic Rep.', 'Country Name'] = 'Iran'
socioeco_df.loc[socioeco_df['Country Name'] == 'Lao PDR', 'Country Name'] = 'Laos'
socioeco_df.loc[socioeco_df['Country Name'] == 'Myanmar', 'Country Name'] = 'Burma'
socioeco_df.loc[socioeco_df['Country Name'] == 'Korea, Rep.', 'Country Name'] = 'South Korea'
socioeco_df.loc[socioeco_df['Country Name'] == 'Kyrgyz Republic', 'Country Name'] = 'Kyrgyzstan'
socioeco_df.loc[socioeco_df['Country Name'] == 'Macao SAR, China', 'Country Name'] = 'Macau'
socioeco_df.loc[socioeco_df['Country Name'] == 'Slovak Republic', 'Country Name'] = 'Slovakia'
socioeco_df.loc[socioeco_df['Country Name'] == 'Turkiye', 'Country Name'] = 'Turkey'
socioeco_df.loc[socioeco_df['Country Name'] == 'United States', 'Country Name'] = 'US'
socioeco_df.loc[socioeco_df['Country Name'] == 'Yemen, Rep.', 'Country Name'] = 'Yemen'

In [ ]:
# pip install fuzzywuzzy[speedup]

In [47]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [48]:
#Standardize the country names
socioeco_df['Clean_Country_Name'] = socioeco_df['Country Name'].str.replace(' ', '').str.lower()
annual_data['Clean_Country'] = annual_data['Country_Region'].str.replace(' ', '').str.lower()

#A function that evaluates text similarity and match the dataframes accordingly
def get_best_match(row, choices):
    best_match = process.extractOne(row['Clean_Country_Name'], choices, scorer=fuzz.token_sort_ratio)
    return best_match[0] if best_match[1] > 80 else None

#Apply to the socioeconomic data
socioeco_df['Best_Match'] = socioeco_df.apply(get_best_match, axis=1, choices=annual_data['Clean_Country'].tolist())

#Merge the dataset
plot2_df = pd.merge(socioeco_df, annual_data, left_on='Best_Match', right_on='Clean_Country', how='left')

# Remove rows where 'Best_Match' is None (NaN)
plot2 = plot2_df.dropna(subset=['Best_Match']).reset_index()

#Remove unnecessary columns
plot2 = plot2.drop(columns=['Clean_Country', "Country_Region", "Clean_Country_Name", "index", "Best_Match"])

#Final dataset for plot2
plot2.head()

,Country Name,Current health expenditure per capita (current US$),"Death rate, crude (per 1,000 people)",GDP per capita (constant 2015 US$),"Life expectancy at birth, total (years)","Literacy rate, adult total (% of people ages 15 and above)",Net migration,Poverty headcount ratio at national poverty line (% of population),"Unemployment, total (% of total labor force)",Population,Density (P/Km²),Confirmed,Deaths,Recovered,Active
0,Afghanistan,74.23410797,6.791,559.140953983718,63.565,NaN,-8082,NaN,11.224,38928346.0,60.0,8370341.0,292432.0,5904418.0,2165900.0
1,Albania,367.75839233,8.48,4543.38771048312,79.282,NaN,-8889,21.8,11.466,2877797.0,105.0,3693101.0,87375.0,1980336.0,1625390.0
2,Algeria,235.99041748,4.392,4737.12977369971,76.474,NaN,-36386,NaN,12.285,43851044.0,18.0,10439282.0,372954.0,7041511.0,3024717.0
3,Samoa,NaN,NaN,13288.3565565116,NaN,NaN,-1650,NaN,NaN,198414.0,70.0,50.0,0.0,50.0,0.0
4,Andorra,3026.59741211,3.9,39413.790877787,NaN,NaN,1238,NaN,NaN,77265.0,164.0,678294.0,15333.0,550059.0,112902.0


In [52]:
plot2.to_csv('plot2.csv', index=False)
files.download('plot2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>